## FDW Crop Production Data Profiling - Somalia

In [1]:
import os, sys, glob, json
from itertools import product, compress, chain
from functools import reduce
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
from tools import save_hdf, PrintAdminUnits, PlotAdminShapes
from tools import FDW_PD_Sweeper, FDW_PD_AvalTable, FDW_PD_Compiling, FDW_PD_GrainTypeAgg, FDW_PD_ValidateFnidName
from tools import FDW_PD_CreateAdminLink, FDW_PD_RatioAdminLink, FDW_PD_ConnectAdminLink
from tools_graphic import PlotBarProduction, PlotLinePAY, PlotHeatCropSystem, PlotHeatSeasonData
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None

In [2]:
# CPCV2 grain code ------------------------------ #
grain_code = pd.read_hdf('./data/crop/grain_cpcv2_code.hdf')
product_category = grain_code[['product', 'product_category']].set_index('product').to_dict()['product_category']
# ----------------------------------------------- #

# Load FEWS NET administrative boundaries ------- #
epsg = 'epsg:32639'
fn_shapes = sorted(glob.glob('./data/shapefile/fewsnet/SO_Admin?_????.shp'))
shape_all = []
for fn in fn_shapes:
    name = fn[-18:-4]
    exec('%s = gpd.read_file("%s").to_crs("%s")' % (name, fn, epsg))
    exec('%s["area"] = %s["geometry"].area/10**6' % (name, name))
    exec('shape_all.append(%s)' % (name))
shape_all = pd.concat(shape_all, axis=0).reset_index(drop=True)
PrintAdminUnits(shape_all)
# ----------------------------------------------- #

# FDW API host address -------------------------- #
host = 'https://fdw.fews.net'
auth = tuple(json.loads(open('token.json', "r").read()))
parameters = {
    'format': 'json',
    'country': 'Somalia',
    'product': 'R011',
    'survey_type': 'crop:best'
}
endpoint = '/api/cropproductionindicatorvalue/'
response = requests.get(host + endpoint, auth=auth, params=parameters, proxies={})
response.raise_for_status()
df = pd.DataFrame.from_records(response.json())
df_origin = df.copy()
# ----------------------------------------------- #

# Manual Pre-processing before Sweeping --------- #
# 1. Default setting 
# a) None-type population group
df.loc[df['population_group'].isna(), 'population_group'] = 'none'
df.loc[df['population_group'] == '', 'population_group'] = 'none'
# 2. Manual setting
# a) Add names to reporting units 
reporting_unit = df.loc[df['fnid'].map(lambda x: x[6] != 'A'), 'fnid'].unique()
df.loc[df.fnid.isin(reporting_unit),'name'] = df.loc[df.fnid.isin(reporting_unit), 'locality_name'].map(lambda x: x.split(' ')[0])
# b) Delay one year of Deyr season data 
# This is an ongoing discussion. Currently we, suspect that the Deyr season data is not year off.
# For example, the 2019-02-01 data corresponds to 2020-02-01 data.
idx_deyr = df['season_name'] == 'Deyr'
delayed = pd.to_datetime(df.loc[idx_deyr, 'season_date']) + pd.offsets.DateOffset(years=1)
df.loc[idx_deyr, 'season_date'] = delayed.dt.strftime('%Y-%m-%d')
# c) Control population groups
# Data of Awdal (SO1990A111) prior to 2017 has a population group (Borama (SO1990A21109) and Baki (SO1990A21102))
# This is not duplicated data, so we will link the data except for one duplicated data point:
df = df[df['id'] != 27061759]
# ----------------------------------------------- #

# FDW Production Data Inspection ---------------- #
df, df_raw = FDW_PD_Sweeper(df)
table_dict = FDW_PD_AvalTable(df, shape_all)
# ----------------------------------------------- #

# FEWS NET Shapefile comparison ----------------- #
shape_used = pd.concat([SO_Admin1_1990, SO_Admin2_1990], axis=0)
PlotAdminShapes(shape_used, label=True)
# ----------------------------------------------- #

- FEWS NET admin shapefiles ------------------- #
        Admin1  # units    Admin2  # units
year                                      
1990  SO1990A1       18  SO1990A2       74
----------------------------------------------- #
- Remove missing records ---------------------- #
Orignial data points: 17,043
Removed 163 "Missing Historic Data" points
Removed 2,927 "Not Collected" points
4,102/4,261 "Area Harvested" points are retained.
1,793/4,260 "Area Planted" points are retained.
4,055/4,261 "Quantity Produced" points are retained.
4,003/4,261 "Yield" points are retained.
Current data points: 13,953

- Minor changes are applied.. ----------------- #

- Basic information --------------------------- #
Data period: 1982 - 2021
3 grain types are found: Maize (Corn), Rice (Paddy), Sorghum
4 seasons are found: Gu (07-01), Deyr (10-01), Gu off-season (09-01), Deyr off-season (03-01)
3 crop production system are found: agro_pastoral, none, riverine
Data sources include:
[1] FSNAU, Somalia ---

- Somalia crop seasonal calendar </br>

![FEWS NET](https://fews.net/sites/default/files/styles/large/public/seasonal-calendar-somalia.png?itok=6LQ1arWc)

- FDW data consists of `SO1990A1` and `SO1990A2`.

| Year | Admin-1 | # units | Admin-2  | # units |
| :--- | :----:  | :----:  | :----:   | ---:    |
| 1990 | SO1990A1| 18      | **`SO1990A2`** | 74      |

- Comparison between admin boundaries </br>

![image](https://github.com/chc-ucsb/GlobalCropData/blob/main/figures/SO_admin_shapes.png?raw=true)

<!-- - Most of reliable data are admin-2 level, and FDW has few data for admin-1 level. Therefore, we aggregate/merge admin-2 level data to represent admin-1 level. -->
- Afmadow (SO1990A22802) has two crop reporting units: Afmadow (SO1990R22805) and Afmadow (Hagar) (SO1990R22806). This is confirmed by FEWS NET crop reporting unit (./data/admin_boundaries/SO_Crop_Units_1990.shp). Since SO1990A22802 has only one off-season record, we aggregate SO1990R22805 and SO1990R22806 to replace SO1990A22802.

| District |Crop reporting units|
| :---:|:---:|
|Afmadow (SO1990A22802) | Afmadow (SO1990R22805) and  Hagar (SO1990R22806)|

- **`SO1990A2`** is used to represent the current admin-level 2 crop data.
- Somalia has four crop seasons: `Deyr`, `Gu`, `Gu off-season`, `Deyr off-season`.
- Somalia has three crop production systems: `agro_pastoral`, `riverine`, `None`.
- Somalia has no population group(s).

In Admin-2 level data, we found unrealistic records. So, we manually remove those records:
- 26 districts have the exactly same `1 mt/ha` yield values in `Deyr` season of `2001`.
- Three districts have unrealistic yield values in `Gu` season of `2005`: Bu'aale (11.65 mt/ha), Jilib (2.27 mt/ha), and Saakow (20.8 mt/ha).

In [3]:
# Define the latest shapefile ------------------- #
latest_level = 2
shape_latest = SO_Admin2_1990.copy().to_crs('epsg:4326')
# ----------------------------------------------- #

# Validation of FNIDs and Names ----------------- #
df = FDW_PD_ValidateFnidName(df, shape_used, shape_latest)
# ----------------------------------------------- #

# FDW Production Data Compiling ----------------- #
area, prod = FDW_PD_Compiling(df, shape_used)
area_all, prod_all = area.copy(), prod.copy()
mdx_pss = area.columns.droplevel([0,1]).unique()
# ----------------------------------------------- #

In [4]:
# Link admin boundaries ------------------------- #
link, over = FDW_PD_CreateAdminLink(SO_Admin2_1990, SO_Admin2_1990, 'ADMIN2', 'ADMIN2', area, epsg)
# Crop specific ratios
link_ratio = FDW_PD_RatioAdminLink(link, area, over, mdx_pss)
# Add current unit to link_ratio
for fnid_new in link_ratio.keys():
    link_ratio[fnid_new][fnid_new] = 1.0
    link_ratio[fnid_new] = link_ratio[fnid_new].sort_index(axis=1, ascending=False)
# Manual Editing
link_ratio['SO1990A22802'][['SO1990R22805', 'SO1990R22806']] = 1.0
# Connect data with AdminLink
area_new, prod_new = FDW_PD_ConnectAdminLink(link_ratio, area, prod, validation=False)
# ----------------------------------------------- #

# Aggregate grain data by grain type ------------ #
[area_new, prod_new, area_all, prod_all] = FDW_PD_GrainTypeAgg([area_new, prod_new, area_all, prod_all], product_category)
# ----------------------------------------------- #

# Manual correction ----------------------------- #
crop_new = prod_new/area_new
# - Remove data before 1995
area_new = area_new.loc[1995:]
prod_new = prod_new.loc[1995:]
# - Unrealistic values in 2005 in Gu season
target = ['SO1990A22701','SO1990A22702','SO1990A22703']
area_new.loc[2005,pd.IndexSlice[target,:,'Maize','Gu',:]] = np.nan
prod_new.loc[2005,pd.IndexSlice[target,:,'Maize','Gu',:]] = np.nan
# # - Districts have the same 1 mt/ha yield records in 2001 in Deyr season
# temp = crop_new.loc[2001, pd.IndexSlice[:,:,'Maize','Deyr']]
# target = temp.index.get_level_values(0)[temp == 1]
# area_new.loc[2001,pd.IndexSlice[target,:,'Maize','Deyr',:]] = np.nan
# prod_new.loc[2001,pd.IndexSlice[target,:,'Maize','Deyr',:]] = np.nan
# # - Unrealistic values in 1995 in Dery season
# target = ['SO1990A22605']
# area_new.loc[1995,pd.IndexSlice[target,:,'Maize','Deyr',:]] = np.nan
# prod_new.loc[1995,pd.IndexSlice[target,:,'Maize','Deyr',:]] = np.nan
# ----------------------------------------------- #

# Complete long format DataFrame
df_area = area_new.T.stack().reset_index().rename({0:'value'},axis=1)
df_area['indicator'] = 'area'
df_prod = prod_new.T.stack().reset_index().rename({0:'value'},axis=1)
df_prod['indicator'] = 'production'
df_yield = (prod_new/area_new).T.stack().reset_index().rename({0:'value'},axis=1)
df_yield['indicator'] = 'yield'
stack = pd.concat([df_area, df_prod, df_yield], axis=0)
# Insert a country name
stack['country'] = 'Somalia'
stack = stack[['fnid','country','name','product','year','season_name','season_date','indicator','value']]
stack = stack.reset_index(drop=True)
# Change season_date to harvest_end
stack.rename(columns={'season_date':'harvest_end'},inplace=True)
stack['harvest_end'] = stack['harvest_end'].replace({
    '07-01':'07-01', # Gu
    '02-01':'02-01', # Deyr
    '09-01':'09-01', # Gu off-season
    '03-01':'03-01', # Deyr off-season
})

# Save data
save_hdf('./data/crop/adm_crop_production_raw_SO.hdf', df)
save_hdf('./data/crop/adm_crop_production_SO.hdf', stack)

CBR is considered for 'SO1990A21604' as no record found in: ['SO1990A21604']
CBR is considered for 'SO1990A21202' as no record found in: ['SO1990A21202']
CBR is considered for 'SO1990A21102' as no record found in: ['SO1990A21102']
CBR is considered for 'SO1990A21703' as no record found in: ['SO1990A21703']
CBR is considered for 'SO1990A21802' as no record found in: ['SO1990A21802']
CBR is considered for 'SO1990A21103' as no record found in: ['SO1990A21103']
CBR is considered for 'SO1990A21603' as no record found in: ['SO1990A21603']
CBR is considered for 'SO1990A21605' as no record found in: ['SO1990A21605']
CBR is considered for 'SO1990A21104' as no record found in: ['SO1990A21104']
CBR is considered for 'SO1990A21601' as no record found in: ['SO1990A21601']
CBR is considered for 'SO1990A21503' as no record found in: ['SO1990A21503']
CBR is considered for 'SO1990A21502' as no record found in: ['SO1990A21502']
CBR is considered for 'SO1990A21606' as no record found in: ['SO1990A21606']

## Visualization of production data

In [5]:
# Bar chart of national grain production
country_iso, country_name = 'SO', 'Somalia'
df = pd.read_hdf('./data/crop/adm_crop_production_%s.hdf' % country_iso)
product_order = ['Maize','Sorghum','Rice']
for season_name in ['Gu','Deyr']:
    footnote = 'National grain production in %s - %s' % (country_name, season_name)
    fn_save = './figures/%s_bar_natgrainprod_%s.png' % (country_iso, season_name)
    sub = df[df['season_name'] == season_name]
    fig = PlotBarProduction(sub, product_order, footnote, fn_save)
    # fig.show()

./figures/SO_bar_natgrainprod_Gu.png is saved.
./figures/SO_bar_natgrainprod_Deyr.png is saved.


![image](https://github.com/chc-ucsb/GlobalCropData/blob/main/figures/SO_bar_natgrainprod_Gu.png?raw=true)
![image](https://github.com/chc-ucsb/GlobalCropData/blob/main/figures/SO_bar_natgrainprod_Deyr.png?raw=true)

In [6]:
# Lineplot of Production-Area-Yield (PAY) time-series
country_iso, country_name = 'SO', 'Somalia'
df = pd.read_hdf('./data/crop/adm_crop_production_%s.hdf' % country_iso)
product_season = [
    ['Maize','Gu'],
    ['Maize','Deyr'],
    ['Sorghum','Gu'],
    ['Sorghum','Deyr'],    
]
for product_name, season_name in product_season:
    footnote = 'Production-Area-Yield (PAY) time-series of %s - %s - %s' % (country_iso, product_name, season_name)
    fn_save = './figures/%s_line_pay_%s_%s.png' % (country_iso, product_name, season_name)
    sub = df[(df['product'] == product_name) & (df['season_name'] == season_name)]
    fig = PlotLinePAY(sub, footnote, fn_save)
    # fig.show()

./figures/SO_line_pay_Maize_Gu.png is saved.
./figures/SO_line_pay_Maize_Deyr.png is saved.
./figures/SO_line_pay_Sorghum_Gu.png is saved.
./figures/SO_line_pay_Sorghum_Deyr.png is saved.


![image](https://github.com/chc-ucsb/GlobalCropData/blob/main/figures/SO_line_pay_Maize_Gu.png?raw=true)
![image](https://github.com/chc-ucsb/GlobalCropData/blob/main/figures/SO_line_pay_Maize_Deyr.png?raw=true)
![image](https://github.com/chc-ucsb/GlobalCropData/blob/main/figures/SO_line_pay_Sorghum_Gu.png?raw=true)
![image](https://github.com/chc-ucsb/GlobalCropData/blob/main/figures/SO_line_pay_Sorghum_Deyr.png?raw=true)

In [7]:
# Heatmap of seasonal data availability
country_iso, country_name = 'SO', 'Somalia'
df = pd.read_hdf('./data/crop/adm_crop_production_raw_%s.hdf' % country_iso)
code = {'Gu':1,'Deyr':10}
comb = {1:1,10:2,11:3,12:3,20:3,21:3,22:3}
comb_name = {1:'Gu',2:'Deyr',3:'Gu + Deyr'}
for product_name in ['Maize (Corn)','Sorghum']:
    data = df[(df['product'] == product_name) & (df['season_name'].isin(code.keys()))]
    footnote = 'Seasonal data availability in %s - %s (uncorrected)' % (country_name, product_name)
    fn_save = './figures/%s_heat_seasondata_%s.png' % (country_iso, product_name)
    fig = PlotHeatSeasonData(data, code, comb, comb_name, footnote, fn_save)
    # fig.show()

./figures/SO_heat_seasondata_Maize (Corn).png is saved.
./figures/SO_heat_seasondata_Sorghum.png is saved.


![image](https://github.com/chc-ucsb/GlobalCropData/blob/main/figures/SO_heat_seasondata_Maize%20(Corn).png?raw=true)
![image](https://github.com/chc-ucsb/GlobalCropData/blob/main/figures/SO_heat_seasondata_Sorghum.png?raw=true)

In [8]:
# Heatmap of crop production system(s) per crop type
country_iso, country_name = 'SO', 'Somalia'
df = pd.read_hdf('./data/crop/adm_crop_production_raw_%s.hdf' % country_iso)
product_season = [
    ['Maize (Corn)', 'Gu'],
    ['Maize (Corn)', 'Deyr'],
]
code = {'none':1,'riverine':10,'agro_pastoral':100}
comb = {1:1,10:2,11:3,100:4,101:5,110:6,111:7}
comb_name = {1:'None',2:'Riverine',3:'Riverine + None',4:'Agro_pastoral',5:'Agro_pastoral + None',6:'Agro_pastoral + Riverine',7:'All'}
for product_name, season_name in product_season:
    data = df[(df['product'] == product_name) & (df['season_name'] == season_name)]
    footnote = 'Reported crop production system(s) in %s - %s - %s (uncorrected)' % ('Somalia', product_name, season_name)
    fn_save = './figures/%s_heat_cropsystem_%s_%s.png' % (country_iso, product_name, season_name)
    fig = PlotHeatCropSystem(data, code, comb, comb_name, footnote, fn_save)
    # fig.show()

./figures/SO_heat_cropsystem_Maize (Corn)_Gu.png is saved.
./figures/SO_heat_cropsystem_Maize (Corn)_Deyr.png is saved.


![image](https://github.com/chc-ucsb/GlobalCropData/blob/main/figures/SO_heat_cropsystem_Maize%20(Corn)_Gu.png?raw=true)
![image](https://github.com/chc-ucsb/GlobalCropData/blob/main/figures/SO_heat_seasondata_SorghumSO_heat_cropsystem_Maize%20(Corn)_Deyr.png?raw=true)

### Irrigated (or riverine) districts

In [9]:
from tools_graphic import PlotMapCropSystem
country_iso, country_name = 'SO', 'Somalia'
df = pd.read_hdf('./data/crop/adm_crop_production_raw_%s.hdf' % country_iso)
product_season = [
    ['Maize (Corn)', 'Gu'],
    ['Maize (Corn)', 'Deyr'],
    ['Sorghum', 'Gu'],
    ['Sorghum', 'Deyr']
]
for product_name, season_name in product_season:
    sub = df[(df['product'] == product_name)&(df['season_name'] == season_name)]
    footnote = 'Crop production system(s) in %s - %s - %s' % ('Somalia', product_name, season_name)
    fn_save = '%s_map_cropsystem_%s_%s.png' % (country_iso, product_name, season_name)
    fig = PlotMapCropSystem(sub, footnote, fn_save)
    # fig.show()

SO_map_cropsystem_Maize (Corn)_Gu.png is saved.
SO_map_cropsystem_Maize (Corn)_Deyr.png is saved.
SO_map_cropsystem_Sorghum_Gu.png is saved.
SO_map_cropsystem_Sorghum_Deyr.png is saved.


![image](https://github.com/chc-ucsb/GlobalCropData/blob/main/figures/SO_map_cropsystem_Maize%20(Corn)_Gu.png?raw=true)
![image](https://github.com/chc-ucsb/GlobalCropData/blob/main/figures/SO_map_cropsystem_Maize%20(Corn)_Deyr.png?raw=true)
![image](https://github.com/chc-ucsb/GlobalCropData/blob/main/figures/SO_map_cropsystem_Sorghum_Gu.png?raw=true)
![image](https://github.com/chc-ucsb/GlobalCropData/blob/main/figures/SO_map_cropsystem_Sorghum_Deyr.png?raw=true)